# Table of Contents

* [4 General Functions (Review)](#general_functions)
    * [4.1 Functions to Save and Open Variables](#open_save)
* [5 Document Clustering](#document_clustering) 
    * [5.1 k-Means Clustering](#k_means)
    * [5.2 Density-based Spatial Clustering of Applications with Noise (DBSCAN)](#dbscan)
    * [5.3 Balanced Iterative Reducing and Clustering using Hierarchies (Birch)](#birch)
    * [5.4 Affinity Propagation](#prop)

# General Functions <a id='general_functions'></a>

## Functions to Save and Open Variables <a id='open_save'></a>

Since it is not uncommon for a machine learning task to take a long time it is good practice to save variables that may be needed in the future. This can be achieved by using the [pickle](https://docs.python.org/3/library/pickle.html) module. This package allows a variable up to 4gb to be saved. This limitation is why the 'metrics' variables are saved as individual items instead of a dictionary.

In [2]:
# Save variables to file
import pickle

def save_var(variable_name):
    """ Saves the variable with the provided variable name 
         in the global namespace to the ./vars folder 
         with the provided same name """
    
    with open('./vars/' + variable_name,'wb') as my_file_obj:
        pickle.dump(globals()[variable_name], my_file_obj, protocol=pickle.HIGHEST_PROTOCOL)

def save_var_list(variable_name_list):
    """ Saves each variable with the provided variable name 
         in the global namespace to the ./vars folder 
         with the provided same name """
    for name in variable_name_list:
        with open('./vars/' + name,'wb') as my_file_obj:
            pickle.dump(globals()[name], my_file_obj, protocol=pickle.HIGHEST_PROTOCOL)

def open_var(file_name):
    """ Returns the variable saved with the provided 
         file name located in the ./vars folder"""
    
    file_object = open('./vars/' + file_name,'rb')  

    loaded_var = pickle.load(file_object)
    
    return loaded_var

def open_var_list(file_name_list):
    """ Loads a variable corresponding to each file name
         in file_name_list to the global namespace. """
    
    for file_name in file_name_list:
        globals()[file_name] = open_var(file_name)

In [3]:
%time mnist = open_var('mnist')

CPU times: user 12 ms, sys: 52 ms, total: 64 ms
Wall time: 61.1 ms


In [ ]:
%%time
# Load Datasets
class Dataset_Part:
    """ Represents a dataset with attributes
         data and target """
    
    data = None
    target = None
    def __init__(self, X, y):
        self.data = X
        self.target = y

open_var_list(['mnist_train', 'mnist_test', 'rcv1_train', 'rcv1_test'])

In [4]:
import matplotlib
import matplotlib.pyplot as plt

def print_digit(dataset, index):
    # Get a random document
    digit_arr = dataset.data[index]
    # Reshape it to the size of the image
    digit_image = digit_arr.reshape(28,28)

    # Some information
    print(f'\tIndex: {index}\tLabel: {dataset.target[index]:.0f}')
    # Show the image
    plt.imshow(digit_image, cmap=matplotlib.cm.binary, interpolation="nearest")
    plt.axis("off")
    plt.show()

# [Document Clustering](http://scikit-learn.org/stable/modules/generated/sklearn.cluster.KMeans.html#sklearn.cluster.KMeans) <a id='document_clustering'></a>

Clustering is an unsupervised training method, meaning it is performed on data without labels. Because of this unsupervised learning is capable of finding relations that may not have been previously observed. 

## [Cluster Evaluation](http://scikit-learn.org/stable/modules/clustering.html#homogeneity-completeness-and-v-measure) <a id='cluster_evaluation'></a>

Unsupervised learning uses different evaluation metrics than supervised learning. This is because unsupervised learning makes assumptions with no prior knowladge (ie. no labels). Since the data does not conform to predetermined labels evaluation metrics such as precision and recall cannot be performed. Instead the following metrics can be used.

+ __homogeneity score__ - A clustering result satisfies homogeneity if all of its clusters contain only data points which are members of a single class. A value of 1.0 represents perfectly homogenious labeling.  
$$ h = 1 - \frac{H(C \mid K)}{H(C)} $$   
$$H(C \mid K) = - \sum^{\mid C \mid}_{c=1} \sum^{\mid K \mid}_{k=1} \frac{n_{c,k}}{n} \cdot \log(\frac{n_{c,k}}{n_k})$$  
$$H(C) = - \sum^{\mid C \mid}_{c=1} \frac{n_{c}}{n} \cdot \log(\frac{n_{c}}{n})$$
+ __completeness score__ - A clustering result satisfies completeness if all the data points that are members of a given class are elements of the same cluster. A value of 1.0 represents perfectly complete labeling.
$$ c = 1 - \frac{H(K \mid C)}{H(K)} $$
+ [__Rand index__](https://doi.org/10.1007/BF01908075) - Measure of similarity between the predicted and true clusters. Rand Index considers all pairs of samples and counts the number of pairs that are assigned correctly to the same cluster, incorrectly to the same cluster, correctly to seperate clusters, and incorrectly to different clusters.
  + __TODO: PARAPHRASE ASSIGNMENT__
  + If C is the ground truth of class assignment and K the clustering, let us define:
    + a as the number of pairs of elements that are in the same set in C and in the same set in K
    + b as the number of pairs of elements that are in different sets in C and in different sets in K
    + $C^{n_{samples}}_2$ as the total number of possible pairs in the dataset (without ordering)
$$ RI = \frac{a + b}{C^{n_{samples}}_2}$$
+ __adjusted Rand score__ - The Rand Index computes a similarity measure between two clusterings by considering all pairs of samples and counting pairs that are assigned in the same or different clusters in the predicted and true clusterings. 
$$ ARI = \frac{RI - E(RI)}{\max(RI) - E(RI)} $$
  + Note - E(RI) means expected RI, or the RI given random labelings. 
+ __mutual information score__ - The measure of similarity between the predicted and true labels, ignoring permutation. Given two sets of clusters $V$ and $U$. Suppose $U$ has size $i$, denoted as $\mid U \mid = i$, and similarly for $\mid V \mid = j$.
$$ MI(U,V) = \sum^{\mid U \mid}_{i=1} \sum^{\mid V \mid}_{j=1} \frac{\mid U_i \cap V_j \mid}{N} \cdot log ( \frac{N \mid U_i \cap V_j \mid}{ \mid U_i \mid \mid V_j \mid}) $$
+ __adjusted mutual info score__ - The mutual information score adjusted to account for the fact tha mutual information score is typically greater when there are more clusters. The adjusted mutual information score is 1 when the two sets of clusters are the same. Random clustering have an expected adjusted mutual information score near 0.
$$ AMI(U,V) = \frac{MI(U,V) - E(MI(U, V))} {\max(H(U), H(V)) - E(MI(U, V))} $$
+ __V-measure__ - The harmonic mean between homogeneity and completeness.
$$ v = 2 \cdot \frac{ \text{homogeneity} \cdot \text{completeness}}{\text{homogeneity} + \text{completeness}} $$

In [1]:
from sklearn import metrics
from time import time

def bench_clust(estimator_lst, name_lst, data, labels):

    print('%-9s\t%-6s\t%-4s\t%-4s\t%-4s\t%-4s\t%-4s' 
          % ( 'title', 'time', 'homog', 'comp', 'v mes',
              'rand', 'mutu'))
    est_lst = []
    for estimator, name in zip(estimator_lst, name_lst):
        t0 = time()
        estimator.fit(data)
        print('%-9s\t%.2fs\t%.3f\t%.3f\t%.3f\t%.3f\t%.3f'
              % (name, (time() - t0),
                 metrics.homogeneity_score(labels, estimator.labels_),
                 metrics.completeness_score(labels, estimator.labels_),
                 metrics.v_measure_score(labels, estimator.labels_),
                 metrics.adjusted_rand_score(labels, estimator.labels_),
                 metrics.adjusted_mutual_info_score(labels,  estimator.labels_)))

        est_lst += [estimator]
    return estimator

## [k-Means Clustering](http://scikit-learn.org/stable/modules/generated/sklearn.cluster.KMeans.html#sklearn.cluster.KMeans) <a id='k_means'></a>

This algorithm is implemented in the sklearn.cluster.KMeans scikit-learn module. K-means clustering attempts to seperate data into a predetermined, k, number of clusters. The aim is to create clusters with equal variance, thus minimizing inertia, also known as the within-cluster sum of squares. Inertia is defined as:

\begin{equation*}
 \sum_{i=0}^n \min_{\mu_j \in C} (\mid \mid x_j - \mu_i \mid \mid^2)
\end{equation*}


[comment]: <> (need to reword, too close to source)
To find clusters k-Means has a three step process explained by [Zhao et al.](https://doi.org/10.1016/j.neucom.2018.02.072) [1] as:

1. Initialize k centroids, one for each cluster. The most basic way to do this is by picking k random samples.
+ Assign each sample to the closest centroid.
+ Recompute centroids with assignments from previous step.
+ Repeat step 2 and step 3 until convergence

In [6]:
from sklearn import metrics
from time import time

def bench_k_means(estimator_lst, name_lst, data, labels):
    print('%-9s\t%-6s\t%-12s\t%-4s\t%-4s\t%-4s\t%-4s\t%-4s\t%-4s' 
      % ( 'title', 'time', 'inertia', 'homog', 'comp', 'v mes', 'rand', 'mutu', 'silh'))
    for estimator, name in zip(estimator_lst, name_lst):
        t0 = time()
        estimator.fit(data)
        print('%-9s\t%.2fs\t%i\t%.3f\t%.3f\t%.3f\t%.3f\t%.3f\t%.3f'
              % (name, (time() - t0), estimator.inertia_,
                 metrics.homogeneity_score(labels, estimator.labels_),
                 metrics.completeness_score(labels, estimator.labels_),
                 metrics.v_measure_score(labels, estimator.labels_),
                 metrics.adjusted_rand_score(labels, estimator.labels_),
                 metrics.adjusted_mutual_info_score(labels,  estimator.labels_),
                 metrics.silhouette_score(data, estimator.labels_,
                                          metric='euclidean',
                                          sample_size=1000)))

In [6]:
from sklearn.cluster import KMeans

bench_k_means([KMeans(init='k-means++', n_clusters=3, n_init=10, n_jobs=-1),
               KMeans(init='k-means++', n_clusters=5, n_init=10, n_jobs=-1),
               KMeans(init='k-means++', n_clusters=10, n_init=10, n_jobs=-1),
               KMeans(init='random', n_clusters=10, n_init=10, n_jobs=-1),
               KMeans(init='k-means++', n_clusters=15, n_init=10, n_jobs=-1),
               KMeans(init='random', n_clusters=15, n_init=10, n_jobs=-1) ],
              ["k-means++ k=3", "k-means++ k=5", "k-means++ k=10", "random k=10", "k-means++ k=15", "random k=15"],
              data=mnist.data, labels=mnist.target)

title    	time  	inertia     	homog	comp	v mes	rand	mutu	silh
k-means++ k=3	17.01s	213604855174	0.211	0.443	0.286	0.172	0.211	0.058
k-means++ k=5	19.18s	197606837877	0.390	0.578	0.465	0.330	0.389	0.067
k-means++ k=10	28.63s	178432235366	0.496	0.503	0.500	0.365	0.496	0.062
random k=10	20.22s	178432527554	0.496	0.503	0.500	0.366	0.496	0.054
k-means++ k=15	24.53s	167394422335	0.577	0.494	0.532	0.380	0.493	0.059
random k=15	32.07s	167392804879	0.579	0.495	0.534	0.382	0.495	0.063


## [Density-based Spatial Clustering of Applications with Noise (DBSCAN)](http://scikit-learn.org/stable/modules/generated/sklearn.cluster.DBSCAN.html#sklearn.cluster.DBSCAN) <a id='dbscan'></a>

The DBSCAN algorithm clusters samples into areas of high density with surrounding low dennsity areas. Because of this Clusters can be any shape and the number of clusters is not predeturmined. Clusters are formed by finding region that satisfy a minimum density, number of documents per area. The shape of the cluster is determined by the distance metric used. Any distance function can be used and the distance function will determine the shape of the clusters [2].

To form a cluster DBSCAN searches for areas with a minimum number of points within a specified distance, $\varepsilon$, from a central point, this area is called an $\varepsilon$-neighborhood. Each point in a $\varepsilon$-neighborhood will expand outward, and if this neighborhood meets the minimum number of points required the cluster is updated to include this $\varepsilon$-neighborhood. Points that are not within $\varepsilon$ of the center, but it is included in the cluster it is said to be density-reachable. [Good visualization here](https://cse.buffalo.edu/~jing/cse601/fa12/materials/clustering_density.pdf).

In [6]:
import numpy as np
import random
ones = np.where(mnist.target == 2.)[0]
result = []
tot_result = 0.
count = 0
# result = np.linalg.norm(mnist.data[ones], 'fro')
for k in ones:
    for i in ones[random.sample(range(len(ones)), 100)]:
        if k != i:
            res = np.linalg.norm(mnist.data[i] - mnist.data[k])
            result += [res]
            tot_result += res
            count += 1

avg_result = tot_result / count
avg_result

2422.3517784359951

In [ ]:
from sklearn.cluster import DBSCAN

estimators = bench_clust([DBSCAN(n_jobs=-1),
            DBSCAN(eps=1000, min_samples=10, n_jobs=-1),
            DBSCAN(eps=100, min_samples=10, n_jobs=-1),
            DBSCAN(eps=10, min_samples=100, n_jobs=-1),
            DBSCAN(eps=10, min_samples=1000, n_jobs=-1)],
            ["auto", "1000, 10", "100,10", "10,100", "10,1000"], data=mnist.data, labels=mnist.target)

## [Affinity Propagation](http://scikit-learn.org/stable/modules/generated/sklearn.cluster.AffinityPropagation.html#sklearn.cluster.AffinityPropagation) <a id='prop'></a>

Affinity propagation uses exemplars instead of centroids. This means that instead of finding a centroid affinity propagation works by finding samples that are most representative of the other samples. In addition the number of clusters is not predetermined. The number of clusters is determined by the data. 

[comment]: <> (need to reword, too close to source)
There are three main formulas used in affinity propagation.

1. The similarity of two samples is denoted $s(i,k)$.
2. The responsibility of a sample, $k$, to be an exemplar of sample, $i$. 
  
 \begin{equation*} 
   r(i, k) \leftarrow s(i,k) - \max [a(i, k') + s(i, k') \forall k' \neq k] 
 \end{equation*}

3. The accumulated evidence that sample $i$ should choose sample $k$ to be its exemplar.
 
 \begin{equation*}
   a(i, k) \leftarrow \min [0, r(k,k) + \sum_{i' ~ s.t.~ i' \notin \{i,k\}} r(i',k)]
 \end{equation*}

In [ ]:
from sklearn.cluster import AffinityPropagation

bench_clust([AffinityPropagation()] ,["auto"], data=mnist.data, labels=mnist.target)

title    	time  	inertia     	homog	comp	v mes	rand	mutu	silh


## [Gaussian Mixture Modles (GMM)](http://scikit-learn.org/stable/modules/generated/sklearn.cluster.Birch.html) <a id='birch'></a>

[Paper](https://rdcu.be/XdFp)  
It is a memory-efficient, online-learning algorithm provided as an alternative to MiniBatchKMeans. It constructs a tree data structure with the cluster centroids being read off the leaf. 

## [Balanced Iterative Reducing and Clustering using Hierarchies (Birch)](http://scikit-learn.org/stable/modules/generated/sklearn.cluster.Birch.html) <a id='birch'></a>

[Paper](https://rdcu.be/XdFp)  
It is a memory-efficient, online-learning algorithm provided as an alternative to MiniBatchKMeans. It constructs a tree data structure with the cluster centroids being read off the leaf. 

### Bibtex:

#### K means article [1]
@article{ZHAO2018195,
title = "k-means: A revisit",
journal = "Neurocomputing",
volume = "291",
pages = "195 - 206",
year = "2018",
issn = "0925-2312",
doi = "https://doi.org/10.1016/j.neucom.2018.02.072",
url = "http://www.sciencedirect.com/science/article/pii/S092523121830239X",
author = "Wan-Lei Zhao and Cheng-Hao Deng and Chong-Wah Ngo",
keywords = "Clustering, -means, Incremental optimization"
}

#### DBSCAN article [2] 
@inproceedings{Ester:1996:DAD:3001460.3001507,
 author = {Ester, Martin and Kriegel, Hans-Peter and Sander, J\"{o}rg and Xu, Xiaowei},
 title = {A Density-based Algorithm for Discovering Clusters a Density-based Algorithm for Discovering Clusters in Large Spatial Databases with Noise},
 booktitle = {Proceedings of the Second International Conference on Knowledge Discovery and Data Mining},
 series = {KDD'96},
 year = {1996},
 location = {Portland, Oregon},
 pages = {226--231},
 numpages = {6},
 url = {http://dl.acm.org/citation.cfm?id=3001460.3001507},
 acmid = {3001507},
 publisher = {AAAI Press},
 keywords = {arbitrary shape of clusters, clustering algorithms, efficiency on large spatial databases, handling nlj4-275oise},
 } 

#### V-Measuer article [3]

@inproceedings{rosenberg2007v,
  title={V-measure: A conditional entropy-based external cluster evaluation measure},
  author={Rosenberg, Andrew and Hirschberg, Julia},
  booktitle={Proceedings of the 2007 joint conference on empirical methods in natural language processing and computational natural language learning (EMNLP-CoNLL)},
  year={2007},
  url = {http://aclweb.org/anthology/D/D07/D07-1043.pdf},
}

#### Rand Index article [4]

@Article{Hubert1985,
author="Hubert, Lawrence
and Arabie, Phipps",
title="Comparing partitions",
journal="Journal of Classification",
year="1985",
month="Dec",
day="01",
volume="2",
number="1",
pages="193--218",
abstract="The problem of comparing two different partitions of a finite set of objects reappears continually in the clustering literature. We begin by reviewing a well-known measure of partition correspondence often attributed to Rand (1971), discuss the issue of correcting this index for chance, and note that a recent normalization strategy developed by Morey and Agresti (1984) and adopted by others (e.g., Miligan and Cooper 1985) is based on an incorrect assumption. Then, the general problem of comparing partitions is approached indirectly by assessing the congruence of two proximity matrices using a simple cross-product measure. They are generated from corresponding partitions using various scoring rules. Special cases derivable include traditionally familiar statistics and/or ones tailored to weight certain object pairs differentially. Finally, we propose a measure based on the comparison of object triples having the advantage of a probabilistic interpretation in addition to being corrected for chance (i.e., assuming a constant value under a reasonable null hypothesis) and bounded between {\textpm}1.",
issn="1432-1343",
doi="10.1007/BF01908075",
url="https://doi.org/10.1007/BF01908075"
}